# Lab 3 — clustering

In [1]:
import numpy as np
import sklearn.cluster as cl
import numpy.linalg as la

import json
import pickle
import random as rand
from copy import deepcopy
from collections import Counter

import matplotlib.pyplot as plt

from bokeh.layouts import column, row
from bokeh.plotting import figure, output_notebook,show, ColumnDataSource
from bokeh.models import CustomJS, ColumnDataSource, Slider, HoverTool
from bokeh.charts import Bar
from bokeh.palettes import small_palettes

%matplotlib inline
plt.style.use("ggplot")
output_notebook()

Loading BokehJS ...

## 3.11 Clustering tags

In [2]:
# Data computed in the exercice 3.2
tag2pca = np.load("tag2pca.npy").item()

# Computes the k-means for our data
def clusters(k):
    kmeans = cl.KMeans(k)
    kmeans.fit(list(tag2pca.values()))
    return kmeans.cluster_centers_

def euclideanDist(a,b):
    return la.norm(a-b)

colorsPalette = small_palettes['Set1'][6]

# Cluster data
data = dict(
        x=[],
        y=[],
        clusters=[],
        colors=[],
    )
# We fill the data with the coordinates of all the clusters
# We do one more than needed because we need 
# len(x)=len(y)=len(clusters), otherwise bokeh won't draw all points...
for k in range(2,7):
    data['clusters'].append(clusters(k))
    
# We initialize the coords at 2 clusters
# and with the first two principal directions
for idx, coords in enumerate(data['clusters'][0]):
    data['x'].append(coords[0])
    data['y'].append(coords[1])
    data['colors'].append(colorsPalette[idx])
    
source = ColumnDataSource(data=data)

# Contains coloring information
dataColors = dict(
    colorsPalette=colorsPalette,
    clusterIdx=[[] for x in range(5)],
)

sourceColors=ColumnDataSource(data=dataColors)

# The points of the data set
dataPoints = dict(
        x=[],
        y=[],
        dataCoords=[],
        colors=[],
    )
# We intialize the coords with the
# first two principal directions
for name, coords in tag2pca.items():
    dataPoints['x'].append(coords[0])
    dataPoints['y'].append(coords[1])
    dataPoints['dataCoords'].append(coords)
    # For each value of k, we add the coloring informations
    for idx, cluster in enumerate(dataColors['clusterIdx']):
        cluster.append(np.argmin(list(map(lambda cluster: euclideanDist(cluster,coords),data['clusters'][idx]))))
# We add the colors for k=2
for idx in dataColors['clusterIdx'][0]:
    dataPoints['colors'].append(colorsPalette[idx])

sourcePoints=ColumnDataSource(data=dataPoints)



p = figure(plot_width=600, plot_height=600, title="k-means clustering")

# We draw all the data points
p.circle('x', 'y', size=5,legend="data",color='colors', source=sourcePoints, fill_alpha=0.3, line_color=None)

# We draw the clusters
p.circle('x', 'y', size=20,legend="cluster",color='colors', source=source)

# Callback function when any of the sliders move
def callback(clusters=source, sourcePoints=sourcePoints,sourceColors=sourceColors, window=None):

    k = slider.get('value')
    x = sliderX.get('value')
    y = sliderY.get('value')
    # We ensure that the two directions are not the same
    if x == y:
        y = (x%5)+1
        sliderY['value'] = y
    
    dataColors = sourceColors.get('data')
    colorsPalette = dataColors['colorsPalette'] 
    
    data =clusters.get('data')
    data['x'] = []
    data['y'] = []
    data['colors'] = []
    # We iterate over the cluster with k points
    for idx, coords in enumerate(data['clusters'][k-2]):
        # We add the cluster in the directions given by the sliders
        data['x'].append(coords[x-1])
        data['y'].append(coords[y-1])
        data['colors'].append(colorsPalette[idx])
    clusters.trigger('change')
    
    dataPoints = sourcePoints.get('data')
    dataPoints['x'] = []
    dataPoints['y'] = []
    dataPoints['colors'] = []
    for idx, coords in enumerate(dataPoints['dataCoords']):
        # We add the data in the directions given by the sliders
        dataPoints['x'].append(coords[x-1])
        dataPoints['y'].append(coords[y-1])
        # We find the right color from the coloring information
        dataPoints['colors'].append(colorsPalette[dataColors['clusterIdx'][k-2][idx]])
        
    sourcePoints.trigger('change')

callback = CustomJS.from_py_func(callback)

slider = Slider(start=2, end=5, value=2, step=1, title="Number of clusters",callback=callback)
callback.args['slider'] = slider

sliderX = Slider(start=1, end=5, value=1, step=1, title="X principal direction", callback=callback)
callback.args['sliderX'] = sliderX

sliderY = Slider(start=1, end=5, value=2, step=1, title="Y principal direction", callback=callback)
callback.args['sliderY'] = sliderY

layout = row(p,column(slider, sliderX, sliderY))
show(layout)

The principal directions that separate the cluster well are the tuples (1,5) and (3,4)

## 3.12 Clustering movies

In [3]:
# util methods for unordered equality
def compareListOfSets(list1, list2):
    set1 = set([tuple(x) for x in list1])
    set2 = set([tuple(x) for x in list2])
    return set1 == set2

# Distance function as described in the handout
def distance(A, B):
    jaccard_index = len(A.intersection(B))/len(A.union(B)) 
    return 1 - jaccard_index

# Computes k medoids for a set of set
# Xin is a list of list
def kMedoids(Xin, k, maxSteps=200):
    # We transform the input in a set of set
    X = set(map(frozenset,Xin))
    # We intialize the medoids at random set
    m = rand.sample(X,k)
    mPrev = deepcopy(m)
    # We run until converge or until a maximum number of steps
    for step in range(maxSteps):
        C = [set() for x in range(k)]
        # We assign each point to a cluster
        for x in X:
            idx = np.argmin(list(map(lambda y: distance(x,y), m)))
            C[idx].add(x)
        # We recompute the medoids
        for i in range(k):
            # The biggest medoids possible is if all dist are equal to 1
            # thus we multiply the max value by 2 to be safe
            mini = len(C[i])*2
            for x in C[i]:
                res = np.sum(list(map(lambda y: distance(x,y), C[i])))
                if res < mini:
                    mini = res
                    m[i] = x
        # If we converged, we stop
        if compareListOfSets(m,mPrev):
            return list(map(set,m))
        mPrev = deepcopy(m)
    return list(map(set, m))

In [5]:
# Reading an object from disk.
with open("most-rated.pickle", "rb") as f:
    movies = pickle.load(f, encoding="utf-8")
movieIds = []
for id, movie in movies:
    movieIds.append(id)
    
data = sc.textFile("/ix/ml-20m/movies.txt").map(json.loads)
# We only care about the list of genres of the selected movies
moviesGenre = data.filter(lambda movie: movie['movieId'] in movieIds).map(lambda movie: movie['genres']).collect()

medoids = kMedoids(moviesGenre,2,1000)

# Cluster the movies genres into the medoids
def clusterMovies(movies, medoids):
    k = len(medoids)
    C = [[] for x in range(k)]
    for genres in movies:
        # Find the closest medoid
        idx = np.argmin(list(map(lambda y: distance(set(genres),y), medoids)))
        C[idx].extend(genres)
    return C

clusters = clusterMovies(moviesGenre,medoids)

# Set of all the genres present in the selected movies
allGenres = set(clusters[0]).union(set(clusters[1]))

# We get a map of genre-> number of occurences
occurences = Counter(clusters[0])

# We add every genre not present in the cluster with a count of 0 to haver a prettier graph
occurences.update({k: 0 for k in allGenres.difference(set(occurences.keys()))})

data = dict(
    genre = list(occurences.keys()),
    occ = list(occurences.values()),
)
bar = Bar(data, label='genre',values='occ', legend=None, color='genre', plot_width=450, title='Cluster 1',ylabel='Occurences',palette=small_palettes['Set1'][9])

# Same than previously, but with the second cluster
occurences = Counter(clusters[1])
occurences.update({k: 0 for k in allGenres.difference(set(occurences.keys()))})

data = dict(
    genre = list(occurences.keys()),
    occ = list(occurences.values()),
)
bar2 = Bar(data, label='genre',values='occ', legend=None, color='genre', plot_width=450, title='Cluster 2',ylabel='Occurences',palette=small_palettes['Set1'][9])

show(row(bar,bar2))

We can see that these cluster...